In [39]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import Counter
import seaborn as sns
import datetime as dt
from dateutil.relativedelta import *
from collections import Counter
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None,'display.max_row', None)
# pd.set_option('display.max_rows', 1000)

default_figsize = (15,5)

### Preparation of Dataframes

In [40]:
# import category information
df_ctg = pd.read_csv("../bulk_export_processed/category_groups_cleaned.csv")
df_fd_rd = pd.read_csv("../bulk_export_processed/funding_rounds.csv")
df_org = pd.read_csv("organizations_cleaned.csv")

df_org.head()

,uuid,country_code,region,city,founded_on,cat_pca_0,cat_pca_1,cat_pca_2,cat_pca_3,cat_pca_4
0,cf3a40e6-920e-edfd-569e-371f84e0a4e4,USA,California,San Francisco,2015-01-01,0.0,0.0,0.0,0.0,0.0
1,fa65a572-1621-dd22-57a8-92bb49217ac5,GBR,England,London,2018-01-01,0.0,0.0,0.0,0.0,0.0
2,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,CHN,Zhejiang,Hangzhou,2014-10-01,0.0,0.0,0.0,0.0,0.0
3,f33a3674-ec6b-14ca-16dc-437f280dc10b,USA,Virginia,Reston,2015-01-01,0.0,0.0,0.0,0.0,0.0
4,8fa7fd0d-d5cc-425d-52cc-a2019e7d42a3,USA,California,Santa Monica,2013-08-20,0.0,0.0,0.0,0.0,0.0


In [41]:
# Stardust ver unique keyword search
def unique_keyword_search(df_col: pd.Series, num_selected: int) -> list:
    '''
    Performs unique keyword search on a dataframe's column for its most common keywords

    :param pd.Series df_col: column of a pd.DataFrame (e.g. df['col'])
    :param int num_selected: number of keywords
    :return: list of keywords in decreasing occurrence
    :rtype: list
    '''
    keywords = []
    for entry in df_col:
        words = entry.split(',')
        [keywords.append(word) for word in words]
    sorted_keywords = Counter(keywords).most_common()
    output_keywords = []
    for i in range(num_selected):
        output_keywords.append(sorted_keywords[i][0])
    return output_keywords

In [42]:
# Stardust ver one-hot encoder V2
def onehot_encoder_v2(df: pd.DataFrame, col_name: str, list_selected: list) -> pd.DataFrame:
    '''
    Performs one-hot encoding on a dataframe's column for its values with most occurrences

    :param pd.DataFrame df: dataframe to be processed
    :param str col_name: name of the encoded column
    :param list list_selected: list of most common values
    :return: processed dataframe
    :rtype: pd.DataFrame
    '''
    for item in list_selected:
        df[item] = np.where(df[col_name] == item, 1, 0) # whenever df[col_name] == cat replace it with 1 else 0
    df.drop(col_name, axis=1, inplace=True)

    return df

In [43]:
df_org.head()

,uuid,country_code,region,city,founded_on,cat_pca_0,cat_pca_1,cat_pca_2,cat_pca_3,cat_pca_4
0,cf3a40e6-920e-edfd-569e-371f84e0a4e4,USA,California,San Francisco,2015-01-01,0.0,0.0,0.0,0.0,0.0
1,fa65a572-1621-dd22-57a8-92bb49217ac5,GBR,England,London,2018-01-01,0.0,0.0,0.0,0.0,0.0
2,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,CHN,Zhejiang,Hangzhou,2014-10-01,0.0,0.0,0.0,0.0,0.0
3,f33a3674-ec6b-14ca-16dc-437f280dc10b,USA,Virginia,Reston,2015-01-01,0.0,0.0,0.0,0.0,0.0
4,8fa7fd0d-d5cc-425d-52cc-a2019e7d42a3,USA,California,Santa Monica,2013-08-20,0.0,0.0,0.0,0.0,0.0


In [44]:
"""
fintech_keywords = unique_keyword_search(df_org['category_list'], 30)
df_org['categories'] = df_org['category_list']

onehot_encoder_v2(df_org, 'category_list', fintech_keywords)
df_org.head(50)

org_df = df_org
"""

"\nfintech_keywords = unique_keyword_search(df_org['category_list'], 30)\ndf_org['categories'] = df_org['category_list']\n\nonehot_encoder_v2(df_org, 'category_list', fintech_keywords)\ndf_org.head(50)\n\norg_df = df_org\n"

In [45]:
# read the CSV
df = pd.read_csv("../bulk_export_processed/event_appearances_cleaned.csv")
# change the cols to keep
keep_col = ['participant_uuid', 'participant_name']
df = df[keep_col]

event_count_dict = df['participant_name'].value_counts().to_dict()
df['event_count'] = df['participant_name'].map(event_count_dict)
df.drop_duplicates(inplace=True)
df.sort_values(by='event_count', axis=0, ascending=False, inplace=True)

df.reset_index(inplace=True)
df.drop(axis=1, labels='index', inplace=True)

event_df = df

In [46]:
# read the CSV
df = pd.read_csv("../bulk_export_processed/investment_partners_processed.csv")
# change the cols to keep
keep_col = ['funding_round_uuid','investor_uuid','partner_uuid']
df = df[keep_col]

df = df.merge(df.groupby('funding_round_uuid').agg(investor_list=('partner_uuid',list)).reset_index())

#for index, row in df.iterrows():
#    row[df.columns.get_loc('investor_list')] = np.insert(row[df.columns.get_loc('investor_list')], 0, row[df.columns.get_loc('investor_uuid')], axis=0)
    
#df.drop(labels=['investor_uuid', 'partner_uuid'], axis=1, inplace=True)
df.drop_duplicates(subset='funding_round_uuid', inplace=True)
df.reset_index(inplace=True)
df.drop(axis=1, labels='index', inplace=True)

partners_df = df

In [47]:
# Stardust ver unique keyword search
def unique_keyword_search(df_col: pd.Series, num_selected: int) -> list:
    '''
    Performs unique keyword search on a dataframe's column for its most common keywords

    :param pd.Series df_col: column of a pd.DataFrame (e.g. df['col'])
    :param int num_selected: number of keywords
    :return: list of keywords in decreasing occurrence
    :rtype: list
    '''
    keywords = []
    for entry in df_col:
        words = entry.split(',')
        [keywords.append(word) for word in words]
    sorted_keywords = Counter(keywords).most_common()
    output_keywords = []
    for i in range(num_selected):
        output_keywords.append(sorted_keywords[i][0])
    return output_keywords

In [48]:
# Stardust ver one-hot encoder
def onehot_encoder(df: pd.DataFrame, col_name: str, num_selected: int) -> pd.DataFrame:
    '''
    Performs one-hot encoding on a dataframe's column for its values with most occurrences

    :param pd.DataFrame df: dataframe to be processed
    :param str col_name: name of the encoded column
    :param int num_selected: number of values with most occurrences
    :return: processed dataframe
    :rtype: pd.DataFrame
    '''
    series = df[col_name].value_counts()
    selected_col_name = series.head(num_selected).index.tolist()

    for item in selected_col_name:
        df[col_name + "_" + item] = np.where(df[col_name] == item, 1, 0) # whenever df[col_name] == cat replace it with 1 else 0
        
    df.drop(columns=col_name, inplace=True)
    return df

In [49]:
# Stardust ver one-hot encoder V2
def onehot_encoder_v2(df: pd.DataFrame, col_name: str, list_selected: list) -> pd.DataFrame:
    '''
    Performs one-hot encoding on a dataframe's column for its values with most occurrences

    :param pd.DataFrame df: dataframe to be processed
    :param str col_name: name of the encoded column
    :param list list_selected: list of most common values
    :return: processed dataframe
    :rtype: pd.DataFrame
    '''
    for item in list_selected:
        df[item] = np.where(df[col_name] == item, 1, 0) # whenever df[col_name] == cat replace it with 1 else 0
    df.drop(col_name, axis=1, inplace=True)

    return df

In [50]:
# read the CSV
ppl_df = pd.read_csv("../bulk_export_processed/people_processed.csv")
# change the cols to keep
# keep_col = ['uuid','gender','featured_job_organization_uuid']
keep_col = ['uuid','gender']
ppl_df = ppl_df[keep_col]
# ppl_df.info()

In [51]:
ppl_df = onehot_encoder(ppl_df, 'gender', 2) # male and female
# ppl_df.info()

ppl_df.reset_index(inplace=True)
ppl_df.drop(axis=1, labels='index', inplace=True)

# read the CSV
deg_df = pd.read_csv("../bulk_export_processed/degrees_cleaned.csv")
# change the cols to keep
keep_col = ['person_uuid','degree_type','subject','started_on','completed_on','is_completed']
deg_df = deg_df[keep_col]

In [52]:
deg_df.drop_duplicates(inplace=True)

degree_type_to_drop = (deg_df['degree_type'] == 'unknown') | (deg_df['degree_type'] == 'Unknown') | (deg_df['degree_type'] == 'Specialization') | (deg_df['degree_type'] == 'Certificate') | (deg_df['degree_type'] == 'Certification')
deg_df = deg_df.drop(deg_df[degree_type_to_drop].index)
deg_df = deg_df.drop(deg_df[deg_df['subject'] == 'unknown'].index)

In [53]:
# use keyword to search for relevant degrees
bachelor_keyword = ['Bachelor', 'Degree', 'BS', 'BSc', 'B.S.', 'Bsc', 'B.S',
'BENG', 'BEng', 'B.Eng.', 'Beng', 'B.Eng', 'BE', 'B.E', 'BA', 'Ba', 'B.A.', 'B.A', 'A.B.', 'AB',
'BBA', 'B.B.A.', 'B.B.A', 'B.Tech', 'B.Tech.', 'B.Com.', 'J.D.', 'JD', 'Juris Doctor']

master_keyword = ['Master', 'Postgraduate', 'Graduate', 'MPHIL', 'MPhil', 'Mphil', 'M.Phil.', 'M.phil', 'M.Phil', 'M.S',
'MS', 'MSc', 'M.Sc', 'Msc', 'MENG', 'MEng', 'M.Eng.', 'M.eng.',
'MA', 'M.A', 'MBA', 'M.B.A.', 'M.B.A', 'Mba', 'M.BA.', 'M.Ba.', 'LLM']

phd_keyword = ['PHD', 'Phd', 'PhD', 'P.HD', 'P.Hd', 'P.hd', 'P.H.D', 'Ph.D.', 'Ph.D.', 'PhD']

# categories can be formed from one or more above elementary keywords
# df.dropna(axis=0, subset=['degree_type'], inplace=True)
deg_df['degree_type'].fillna(value='N/A', inplace=True)
deg_df.loc[deg_df['degree_type'].str.contains('|'.join(phd_keyword)),'degree_type']='PhD'
deg_df.loc[deg_df['degree_type'].str.contains('|'.join(master_keyword)),'degree_type']='Master'
deg_df.loc[deg_df['degree_type'].str.contains('|'.join(bachelor_keyword)),'degree_type']='Bachelor'

degree_type = []

# perform ordinal encoding: bachelor = 1, master = 2, PhD = 3, no/others = 0
for degree in deg_df['degree_type']:
    if degree == 'Bachelor':
        degree_type.append(1)
    elif degree == 'Master':
        degree_type.append(2)
    elif degree == 'PhD':
        degree_type.append(3)
    else:
        degree_type.append(0)
        
deg_df['degree_type'] = degree_type

In [54]:
deg_df = onehot_encoder(deg_df, 'subject', 30)

deg_df.reset_index(inplace=True)
deg_df.drop(axis=1, labels='index', inplace=True)

deg_df.sort_values(by='degree_type', ascending=False, inplace=True) # PhD > Master > Bachelor
deg_df = deg_df.drop_duplicates(subset='person_uuid', keep="first") # only consider the highest degree obtained

degree_date = ['started_on', 'completed_on']
has_degree = np.where(deg_df['degree_type'] != 0, 1, 0)

# ignore for now
# for col in degree_date:
#     deg_df[col] = pd.to_datetime(deg_df[col], errors='coerce', format='%Y-%m-%d') # 'coerce' converts NaN to NaT
#     mean = deg_df[col].mean()
#     deg_df.loc[has_degree,col].fillna(value=mean, inplace=True)

deg_df['is_completed'].fillna(value=0, inplace=True)
deg_df['is_completed'] = deg_df['is_completed']*1

deg_df.reset_index(inplace=True)
deg_df.drop(axis=1, labels='index', inplace=True)

In [55]:
# join people and degrees
ppl_join = ppl_df.set_index('uuid').join(deg_df.set_index('person_uuid'))

# some cols will use 0 as the fillna() value
col_nan_to_zero = ['degree_type','is_completed']
for col in col_nan_to_zero:
    ppl_join[col].fillna(value=0, inplace=True)
ppl_join.iloc[9:].fillna(value=0, inplace=True)

ppl_join.reset_index(inplace=True)

/opt/anaconda3/envs/UBS/lib/python3.8/site-packages/pandas/core/frame.py:4321: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [56]:
# read the CSV
df1 = pd.read_csv("../bulk_export_processed/jobs_cleaned_1.csv")
df2 = pd.read_csv("../bulk_export_processed/jobs_cleaned_2.csv")
df3 = pd.read_csv("../bulk_export_processed/jobs_cleaned_3.csv")
job_df = df1.append(df2)
job_df= job_df.append(df3)
job_df= job_df.reset_index()
job_df= job_df.drop(columns=["index"])

# change the cols to keep
keep_col = ['person_uuid','org_uuid','started_on','ended_on','is_current']
job_df = job_df[keep_col]
job_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589222 entries, 0 to 1589221
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   person_uuid  1589222 non-null  object
 1   org_uuid     1589222 non-null  object
 2   started_on   786367 non-null   object
 3   ended_on     293932 non-null   object
 4   is_current   1589222 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 50.0+ MB


In [57]:
job_is_current = job_df['is_current'].array
job_df.loc[job_is_current,'ended_on'] = job_df.loc[job_is_current,'ended_on'].fillna('2020-12-30')
job_df.drop(columns='is_current', inplace=True)

# find the duration of the job
job_df['job_duration'] = pd.to_datetime(job_df['ended_on'], errors='coerce', format='%Y-%m-%d') - pd.to_datetime(job_df['started_on'], errors='coerce', format='%Y-%m-%d')
# take mean for NaN
mean = job_df['job_duration'].mean()
job_df['job_duration'].fillna(value=mean, inplace=True)
# convert TimeDelta to int
job_df['job_duration'] = job_df['job_duration'].apply(lambda x: x.days)

job_df.drop(columns=['started_on','ended_on'], inplace=True)

# join jobs and ppl_join (= ppl + degrees)
job_join = job_df.set_index('person_uuid').join(ppl_join.set_index('uuid'))

job_join.drop(columns=['started_on','completed_on','is_completed'], inplace=True) # may not drop in final ver
job_join.fillna(value=0, inplace=True)

In [58]:
test = job_join.groupby(['org_uuid']).sum()

In [59]:
# read the CSV
df = pd.read_csv("../bulk_export_processed/investors_processed.csv")
# change the cols to keep
keep_col = ['uuid','name','investment_count','founded_on','closed_on']
df = df[keep_col]

In [60]:
df.dropna(axis=0, how='any', subset=['investment_count'], inplace=True)

df.drop(df[df['investment_count'] < 100].index, inplace=True)

df['investment_count'] = df['investment_count'].astype(int)

df.sort_values(by='investment_count', axis=0, ascending=False, inplace=True)

df.reset_index(inplace=True)
df.drop(axis=1, labels='index', inplace=True)

investor_df = df

In [61]:
acq_df = pd.read_csv("../bulk_export_processed/acquisitions_processed.csv")
fund_df = pd.read_csv("../bulk_export_processed/funds_processed.csv") #investors' investment funds
fund_rd_df = pd.read_csv("../bulk_export_processed/funding_rounds.csv") # each funding round in the dataset
ipo_df = pd.read_csv("../bulk_export_processed/ipos_cleaned.csv") 
org_parent_df= pd.read_csv("../bulk_export_processed/org_parents_cleaned.csv") #Mapping between parent organizations and subsidiaries

In [62]:
#All dataframes are now ready, org_df is the master dataframe
org_df.head()

,uuid,country_code,region,city,status,category_groups_list,num_funding_rounds,total_funding_usd,founded_on,last_funding_on,closed_on,employee_count,primary_role,num_exits,categories,Financial Services,FinTech,Finance,Blockchain,Information Technology,Software,Payments,Cryptocurrency,Venture Capital,Internet,Banking,Consulting,Mobile Payments,E-Commerce,Insurance,Bitcoin,Artificial Intelligence,Mobile,Crowdfunding,SaaS,Real Estate,Apps,Personal Finance,Accounting,Mobile Apps,Asset Management,Marketplace,Lending,Big Data,Machine Learning
0,cf3a40e6-920e-edfd-569e-371f84e0a4e4,USA,California,San Francisco,operating,"Financial Services,Health Care,Lending and Inv...",NaN,NaN,2015-01-01,NaN,NaN,101-250,investor,24.0,"Finance,Financial Services,Health Care,Venture...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,fa65a572-1621-dd22-57a8-92bb49217ac5,GBR,England,London,operating,"Financial Services,Lending and Investments",NaN,NaN,2018-01-01,NaN,NaN,11-50,investor,10.0,"Financial Services,FinTech,Venture Capital",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,CHN,Zhejiang,Hangzhou,operating,"Commerce and Shopping,Financial Services,Inter...",4.0,2.200000e+10,2014-10-01,2018-06-08,NaN,5001-10000,company,12.0,"Banking,E-Commerce,Financial Services,FinTech,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,f33a3674-ec6b-14ca-16dc-437f280dc10b,USA,Virginia,Reston,operating,"Commerce and Shopping,Financial Services,Infor...",NaN,NaN,2015-01-01,NaN,NaN,1-10,investor,19.0,"E-Commerce,Financial Services,Information Tech...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8fa7fd0d-d5cc-425d-52cc-a2019e7d42a3,USA,California,Santa Monica,operating,"Financial Services,Information Technology,Priv...",1.0,NaN,2013-08-20,2014-01-23,NaN,1-10,company,NaN,"Cyber Security,Developer APIs,FinTech,Software",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
#fund_rd_df
ppl_df = job_join.drop_duplicates(subset = ["org_uuid"])
fin_tech_ppl_df = ppl_df.loc[ppl_df['org_uuid'].isin(org_df["uuid"])]
ppl_df = fin_tech_ppl_df

### Handling Naming Issues, and dropping some overlapped columns

In [64]:
# Some prefixes are needed due to duplicate coulmn names with other csv
# Some repeated columns existing info in other dataframes can be deleted

acq_df = acq_df.drop(columns=['acquiree_name', 'acquiree_country_code', 'acquiree_region', 'acquiree_city', 'rank'])
acq_df = acq_df.add_prefix('acquisitions_')
fund_df = fund_df.add_prefix('funds_')
ipo_df = ipo_df.add_prefix('ipo_')
fund_rd_df = fund_rd_df.add_prefix('fund_rd_')

org_parent_df = org_parent_df.add_prefix('parent_org_')

### Firstly, handle investor, partners and fundings. As a partner is also an investor himself, two joinings are needed

In [65]:
#Joining Investor and Partners

partner_big = partners_df.loc[partners_df['partner_uuid'].isin(investor_df["uuid"])]
length = len(partner_big)
big_partner = []
for i in range(length):
    big_partner.append(1)
partner_big['any_top_partner'] = big_partner

partner_df = partners_df.loc[partners_df['investor_uuid'].isin(investor_df["uuid"])]
length = len(partner_df)
any_partner = []
for i in range(length):
    any_partner.append(1)
partner_df['any_partner'] = any_partner

partner_df.drop(columns=['partner_uuid', 'investor_list'], inplace=True)
partner_big.drop(columns=['investor_uuid', 'partner_uuid', 'investor_list'], inplace=True)

partner_df = partner_df.set_index('funding_round_uuid').join(partner_big.set_index('funding_round_uuid'))

partner_df.fillna(0)

partner_df['funding_round_uuid'] = partner_df.index
invest_join_df = investor_df.set_index('uuid').join(partner_df.set_index('investor_uuid'))
invest_join_df['any_partner'] = invest_join_df['any_partner'].fillna(0)

<ipython-input-65-0a48392b49d0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner_big['any_top_partner'] = big_partner
<ipython-input-65-0a48392b49d0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner_df['any_partner'] = any_partner
/opt/anaconda3/envs/UBS/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [66]:
# Joining investor and funding rounds
fund_rd_df = fund_rd_df.set_index('fund_rd_uuid').join(invest_join_df.set_index('funding_round_uuid'))
fund_rd_df = fund_rd_df.drop(columns=['fund_rd_raised_amount','fund_rd_raised_amount_currency_code','fund_rd_post_money_valuation',\
                                     'fund_rd_post_money_valuation_currency_code','fund_rd_lead_investor_uuids',\
                                     'name','founded_on','closed_on'])

In [67]:
ipo_df.drop(columns=['ipo_uuid', 'ipo_country_code', 'ipo_region', 'ipo_city','ipo_stock_symbol','ipo_stock_exchange_symbol'],inplace= True)

### Next, we handle the event. Link the event participants to the events

In [68]:
# Joining event and event_app
#event_join_df = event_df.set_index('event_uuid').join(event_app_df.set_index('event_uuid'))

#people_event_df = event_join_df.loc[event_join_df['person'] == 1]
#org_event_df = event_join_df.loc[event_join_df['organization'] == 1]

### After sorting out all dataframes that are replated to "people entity", we are ready to build up a large dataframe that consists of people_uuid as index

In [69]:
#Joining People and Degrees
#ppl_join = ppl_df.set_index('personal_uuid').join(deg_df.set_index('degree_person_uuid'))

In [70]:
#Joining People and People Event Participant 
#ppl_join = ppl_join.join(people_event_df.set_index('participant_uuid'))

In [71]:
#Add prefix to avoid overlap of column names
#ppl_join = ppl_join.add_prefix('person_')

### Now, it is time to handle organizations, put org_uuid as index

In [72]:
#Joining Organizations and fund_rounds
org_join = org_df.set_index('uuid').join(fund_rd_df.set_index('fund_rd_org_uuid'))

In [73]:
#Joining Organizations and IPO
org_join = org_join.join(ipo_df.set_index('ipo_org_uuid'))

In [74]:
#Joining Organizations and funds
org_join = org_join.join(fund_df.set_index('funds_entity_uuid'))

In [75]:
#Joining Organizations and acquisitions
org_join = org_join.join(acq_df.set_index('acquisitions_acquiree_uuid'))

In [76]:
#Joining Organizations and categories
#org_join = org_join.join(cate_gp_df.set_index('cat_uuid'))

In [77]:
#Joining Organizations and its parents, if any
org_join = org_join.join(org_parent_df.set_index('parent_org_uuid'))

In [78]:
#Joining Organizations and Organization Event Participant
org_join = org_join.join(event_df.set_index('participant_uuid'))

### Have a look at the current large dataframes and drop some columns (org_join)

In [79]:
#Drop away emtpy columns to improve efficiency
ppl_join.dropna(how = 'all', axis = 1, inplace = True)
#Drop away emtpy columns to improve efficiency
org_join.dropna(how = 'all', axis = 1, inplace = True)

In [80]:
org_join.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51912 entries, 0001a6ec-e7e9-4d1b-8a77-adb5ac815420 to ffffabce-6d4a-b3d1-13c0-4e90cedf5270
Data columns (total 83 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   country_code                        48289 non-null  object 
 1   region                              48289 non-null  object 
 2   city                                48289 non-null  object 
 3   status                              51912 non-null  object 
 4   category_groups_list                51912 non-null  object 
 5   num_funding_rounds                  25656 non-null  float64
 6   total_funding_usd                   21865 non-null  float64
 7   founded_on                          51912 non-null  object 
 8   last_funding_on                     25652 non-null  object 
 9   closed_on                           820 non-null    object 
 10  employee_count                      51912 non

In [81]:
#Drop away columns that are without any use to improve efficiency
org_join = org_join.drop(columns = ['fund_rd_state_code', 'fund_rd_region', 'fund_rd_city', \
                'acquisitions_uuid', 'parent_org_rank', 'participant_name'])

In [82]:
ppl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20522 entries, 0000bf3d-7645-e2ef-ec7b-0408a9b2f879 to fff7c487-29b5-b598-586a-3b6eb5185e73
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   org_uuid                         20522 non-null  object 
 1   job_duration                     20522 non-null  int64  
 2   gender_male                      20522 non-null  float64
 3   gender_female                    20522 non-null  float64
 4   degree_type                      20522 non-null  float64
 5   subject_Computer Science         20522 non-null  float64
 6   subject_Economics                20522 non-null  float64
 7   subject_Finance                  20522 non-null  float64
 8   subject_Electrical Engineering   20522 non-null  float64
 9   subject_Business Administration  20522 non-null  float64
 10  subject_Mechanical Engineering   20522 non-null  float64
 11  subject_Marketing  

### Finally, it is about the linkage between people and organization, through "job"

In [83]:
#Join Job with People
#job_join = job_df.set_index('job_person_uuid').join(ppl_join)

In [84]:
#Drop away columns that are without any use to improve efficiency
#job_join = job_join.drop(columns=['job_uuid', 'person_participant_name',\
#                                 'person_degree_uuid', 'person_degree_institution_uuid', 'person_event_venue_name',\
#                                 'person_event_short_description', 'person_event_description'])
#job_join = job_join.rename(columns={'job_person_name': 'person_name','person_event_event_roles':'person_event_roles',\
#                                   'event_event_roles':'event_roles', 'event_names':'event_name', 'person_event_names': \
#                                    'person_event_name', 'job_org_name': 'organization_name'})

In [85]:
#Join Organization with Job
org_join = org_join.join(ppl_df.set_index('org_uuid'))

In [86]:
df = org_join
df.iloc[:,:100].info()

<class 'pandas.core.frame.DataFrame'>
Index: 51912 entries, 0001a6ec-e7e9-4d1b-8a77-adb5ac815420 to ffffabce-6d4a-b3d1-13c0-4e90cedf5270
Data columns (total 100 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   country_code                        48289 non-null  object 
 1   region                              48289 non-null  object 
 2   city                                48289 non-null  object 
 3   status                              51912 non-null  object 
 4   category_groups_list                51912 non-null  object 
 5   num_funding_rounds                  25656 non-null  float64
 6   total_funding_usd                   21865 non-null  float64
 7   founded_on                          51912 non-null  object 
 8   last_funding_on                     25652 non-null  object 
 9   closed_on                           820 non-null    object 
 10  employee_count                      51912 no

In [87]:
#Drop away emtpy columns to simplify the final dataframe
org_join.dropna(how = 'all', axis = 1, inplace = True)

### Final preparation for the overall joint dataframe

In [88]:
#Extract the uuid from index and put back into a proper column
org_join['uuid'] = org_join.index
x = org_join.columns.get_loc('ipo_org_name')
#Move the uuid column and the organization name column to the front for easier references
cols = org_join.columns.tolist()
cols = cols[-1:] + cols[x:x+1] + cols[0:x] + cols[x+1:-1]
org_join = org_join[cols]

In [89]:
#Reindex the dataframe so that the index column is not uuid any more, but integer values "0, 1, 2..."
new_index = []
for i in range(len(org_join)):
    new_index.append(i)
org_join.index = new_index

### Have a look at the joint dataframe

In [90]:
#The final DataFrame
org_join = org_join.rename(columns={"ipo_org_name": "company_name","uuid": "company_uuid" })
org_join.iloc[50000:50004]

,company_uuid,company_name,country_code,region,city,status,category_groups_list,num_funding_rounds,total_funding_usd,founded_on,last_funding_on,closed_on,employee_count,primary_role,num_exits,categories,Financial Services,FinTech,Finance,Blockchain,Information Technology,Software,Payments,Cryptocurrency,Venture Capital,Internet,Banking,Consulting,Mobile Payments,E-Commerce,Insurance,Bitcoin,Artificial Intelligence,Mobile,Crowdfunding,SaaS,Real Estate,Apps,Personal Finance,Accounting,Mobile Apps,Asset Management,Marketplace,Lending,Big Data,Machine Learning,fund_rd_name,fund_rd_type,fund_rd_permalink,fund_rd_cb_url,fund_rd_rank,fund_rd_created_at,fund_rd_updated_at,fund_rd_country_code,fund_rd_investment_type,fund_rd_announced_on,fund_rd_raised_amount_usd,fund_rd_post_money_valuation_usd,fund_rd_investor_count,fund_rd_org_name,investment_count,any_partner,any_top_partner,ipo_went_public_on,ipo_share_price_usd,ipo_valuation_price_usd,ipo_money_raised_usd,funds_uuid,funds_raised_amount_usd,acquisitions_acquirer_uuid,acquisitions_acquirer_name,acquisitions_acquirer_country_code,acquisitions_acquirer_region,acquisitions_acquirer_city,acquisitions_acquired_on,acquisitions_price_usd,parent_org_parent_uuid,event_count,job_duration,gender_male,gender_female,degree_type,subject_Computer Science,subject_Economics,subject_Finance,subject_Electrical Engineering,subject_Business Administration,subject_Mechanical Engineering,subject_Marketing,subject_Accounting,subject_Law,subject_Business,subject_Computer Engineering,subject_Physics,subject_Political Science,subject_Management,subject_Engineering,subject_Psychology,subject_Mathematics,subject_Chemical Engineering,subject_Chemistry,subject_History,subject_Biology,subject_International Business,subject_Information Technology,subject_Business Management,subject_English,subject_Industrial Engineering,subject_Civil Engineering,subject_Biochemistry,subject_Medicine,subject_Philosophy
50000,f6aacf80-5b0a-13f7-5713-96f85fe5dd0d,NaN,MEX,Distrito Federal,Mexico City,operating,"Commerce and Shopping,Financial Services",2.0,650000.0,2017-02-08,2019-06-01,NaN,1-10,company,NaN,"Financial Services,FinTech,Marketplace,Wealth ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Seed Round - Invierte con Expertos,funding_round,invierte-con-expertos-seed--62386b67,https://www.crunchbase.com/funding_round/invie...,349055.0,2017-05-04 01:46:15,2018-02-12 23:44:42,MEX,seed,2017-05-02,400000.0,1400000.0,NaN,Invierte con Expertos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2188.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
50001,f6aacf80-5b0a-13f7-5713-96f85fe5dd0d,NaN,MEX,Distrito Federal,Mexico City,operating,"Commerce and Shopping,Financial Services",2.0,650000.0,2017-02-08,2019-06-01,NaN,1-10,company,NaN,"Financial Services,FinTech,Marketplace,Wealth ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Seed Round - Invierte con Expertos,funding_round,invierte-con-expertos-seed--d3417db3,https://www.crunchbase.com/funding_round/invie...,149805.0,2019-07-18 18:52:29,2019-07-18 18:52:29,MEX,seed,2019-06-01,250000.0,1250000.0,NaN,Invierte con Expertos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2188.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
50002,f6aaf00a-ecf9-425d-a617-d0f7929ffa96,NaN,MEX,Distrito Federal,Miguel Hidalgo,operating,Financial Services,1.0,500000.0,2017-08-01,2017-08-16,NaN,1-10,company,NaN,"Finance,Financial Services,FinTech",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Seed Round - LendInc,funding_round,lendinc-seed--a00c21ef,https://www.crunchbase.com/funding_round/lendi...,310777.0,2018-02-26 17:20:49,2018-02-26 17:20:49,MEX,seed,2017-08-16,500000.0,NaN,1.0,LendInc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [91]:
org_join.info()
unique_df = org_join.drop_duplicates(subset = ["company_uuid"])
unique_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51912 entries, 0 to 51911
Columns: 112 entries, company_uuid to subject_Philosophy
dtypes: float64(50), int64(30), object(32)
memory usage: 44.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37198 entries, 0 to 51906
Columns: 112 entries, company_uuid to subject_Philosophy
dtypes: float64(50), int64(30), object(32)
memory usage: 32.1+ MB


In [92]:
org_join.to_csv("unifed_csv_with_multiple_funds.csv", index=False)
unique_df.to_csv("unifed_csv_without_duplicated_company.csv", index=False)

In [93]:
"""
#df.info() for all columns
num_col = len(org_join.columns)
num_col_100 = int(num_col/100)
for i in range (1,num_col_100):
    print(org_join.iloc[:,100*(i-1):100*i].info())
print(org_join.iloc[:,num_col_100:].info())

#import libraries
import matplotlib.pyplot as plt
from datetime import datetime
import datetime

unique_df = org_join.drop_duplicates(subset = ["uuid"])


founded_on = pd.to_datetime(unique_df['founded_on'], infer_datetime_format=True)
today = ['1/1/2021']
today = pd.DataFrame(today)
today = pd.to_datetime(today[0], infer_datetime_format=True)
today = today.apply(lambda x: x.value)
today = today.loc[0]

#Convert from milliseconds and microseconds into unit of year
founded_on = founded_on.apply(lambda x: x.value)
age = founded_on.apply(lambda x: (today-x)/(365*24*3600*1000*1000000))
age = round(age, 2)
"""

'\n#df.info() for all columns\nnum_col = len(org_join.columns)\nnum_col_100 = int(num_col/100)\nfor i in range (1,num_col_100):\n    print(org_join.iloc[:,100*(i-1):100*i].info())\nprint(org_join.iloc[:,num_col_100:].info())\n\n#import libraries\nimport matplotlib.pyplot as plt\nfrom datetime import datetime\nimport datetime\n\nunique_df = org_join.drop_duplicates(subset = ["uuid"])\n\n\nfounded_on = pd.to_datetime(unique_df[\'founded_on\'], infer_datetime_format=True)\ntoday = [\'1/1/2021\']\ntoday = pd.DataFrame(today)\ntoday = pd.to_datetime(today[0], infer_datetime_format=True)\ntoday = today.apply(lambda x: x.value)\ntoday = today.loc[0]\n\n#Convert from milliseconds and microseconds into unit of year\nfounded_on = founded_on.apply(lambda x: x.value)\nage = founded_on.apply(lambda x: (today-x)/(365*24*3600*1000*1000000))\nage = round(age, 2)\n'